# FIT 3139 Assignment 2

## Question proposed

How do factors such as limited resources, predation and germination rate affect the seed yield of a plant species?

## Model extension

I have selected the Lotka-Volterra model to simulate this scenario. Although, the Lotka-Volterra model is usually used for 2 species predator-prey interactions, I have extended it to include a third "species". There is an argument to be made that an SIR model would be more appropriate for this scenario, but I have chosen the Lotka-Volterra model because it is more intuitive and easier to understand given the dynamics of the system.

**The scenario is as follows:**
- Species 1: Seeds
- Species 2: Plants (yield)
- Species 3: Predators (animals that eat the plants)

In the model, the seeds are attempting to grow into plants, but if there are too many seeds in the system, they will compete for the limited resources and many might not grow into plants. Furthermore, the plants are being eaten by the predators, while the seeds/plants are small there are more predators capableof eating them, but as they grow, they become less vulnerable to predation. The predators are also affected by the availability of plants, if there are not enough plants, the predators will starve and die.

**To summarise the above**

- Seeds     - competing with each other and being eaten by predators
- Plants    - being eaten by predators (assuming that after competing during seed phase, there is minimal competition between plants for resources)
- Predators - eating seeds and plants

## Model equations

$$

\begin{aligned}
& Seeds = S \\
& Plants = P \\
& Predators = R \\
\end{aligned}
\\
$$

UsageError: %%latex is a cell magic, but the cell body is empty.
